In [1]:
import pandas as pd
import re
import math
import sklearn
from sklearn.tree import DecisionTreeClassifier 
from sklearn.preprocessing import LabelEncoder

In [2]:
trainSet_1 = pd.read_csv("SBD.train", sep = "\t", header = None)
testSet_1 = pd.read_csv("SBD.test", sep = "\t", header = None)
trainSet_2 = pd.read_csv("SBD.train", sep = "\t", header = None)
testSet_2 = pd.read_csv("SBD.test", sep = "\t", header = None)
trainSet_3 = pd.read_csv("SBD.train", sep = "\t", header = None)
testSet_3 = pd.read_csv("SBD.test", sep = "\t", header = None)

FileNotFoundError: [Errno 2] File SBD.train does not exist: 'SBD.train'

In [33]:
train_1 = trainSet_1[trainSet_1.columns[0]].str.split(" ", n = 2, expand = True) 
test_1 = testSet_1[testSet_1.columns[0]].str.split(" ", n = 2, expand = True)  
train_2 = trainSet_2[trainSet_2.columns[0]].str.split(" ", n = 2, expand = True) 
test_2 = testSet_2[testSet_2.columns[0]].str.split(" ", n = 2, expand = True)  
train_3 = trainSet_3[trainSet_3.columns[0]].str.split(" ", n = 2, expand = True) 
test_3 = testSet_3[testSet_3.columns[0]].str.split(" ", n = 2, expand = True)  

In [34]:
train_1 = train_1.drop(train_1.columns[[0]], axis = 1)
test_1 = test_1.drop(test_1.columns[[0]], axis = 1)

train_2 = train_2.drop(train_2.columns[[0]], axis = 1)
test_2 = test_2.drop(test_2.columns[[0]], axis = 1)

train_3 = train_3.drop(train_3.columns[[0]], axis = 1)
test_3 = test_3.drop(test_3.columns[[0]], axis = 1)

In [35]:
train_1.columns = ['Words', 'labels']
test_1.columns = ['Words', 'labels']

train_2.columns = ['Words', 'labels']
test_2.columns = ['Words', 'labels']

train_3.columns = ['Words', 'labels']
test_3.columns = ['Words', 'labels']


In [36]:
def getTheLeftWord (row):
    if row["Words"].find(".") != -1:
        
        extractWord = row["Words"].split(".")
        if "" in extractWord:
            extractWord.remove("")

        extractWord = extractWord[0]
        return extractWord
    else: return "-"

Features:
1. Words to the left of "."
2. Words to the right of "."
3. Length of L < 3
4. Is R capitalized?
5. Is L capitalized?
The additional features are a below:
6. Is there a " to left?
7. Is there a " to the right?
8. Is R a numeric (float or digit)?


In [37]:
# Feature 1
# Finding words to the left
#edge case: None if 1st row
train_1["L"] = train_1.apply (lambda x: getTheLeftWord(x), axis = 1)
test_1["L"] = test_1.apply (lambda x: getTheLeftWord(x), axis = 1)

train_2["L"] = train_2.apply (lambda x: getTheLeftWord(x), axis = 1)
test_2["L"] = test_2.apply (lambda x: getTheLeftWord(x), axis = 1)

In [38]:
# Feature 2
# Finding words to the Right
train_1["R"] = train_1["Words"].shift(-1, axis = 0) 
test_1["R"] = test_1["Words"].shift(-1, axis = 0) 

train_2["R"] = train_2["Words"].shift(-1, axis = 0) 
test_2["R"] = test_2["Words"].shift(-1, axis = 0) 
# new

In [39]:
# check if we have a word in the first place
def isLessThan3 (row):
    if isinstance(row["L"], str) :
        if (re.search('[a-zA-Z]', row["L"])):
            if (len(row["L"])<3):
                return True
            else:
                return False
    
    return False

In [40]:
# Feature 3
# If L < 3?
train_1["L < 3"] = train_1.apply (lambda x: isLessThan3(x), axis = 1)
test_1["L < 3"] = test_1.apply (lambda x: isLessThan3(x), axis = 1)

train_2["L < 3"] = train_2.apply (lambda x: isLessThan3(x), axis = 1)
test_2["L < 3"] = test_2.apply (lambda x: isLessThan3(x), axis = 1)

In [41]:
# check for punctuation, ensure it is alpha, NaN
def isWordCapitalized (entry):
    if isinstance(entry, str) :
        if (re.search('[a-zA-Z]', entry)):
            if entry[0].isupper():
                return True
            else:
                return False
    return False
    

In [42]:
# Feature 4
# If L is Capitalized
train_1["L Captilaized"] = train_1.apply (lambda x: isWordCapitalized(x["L"]), axis = 1)
test_1["L Captilaized"] = test_1.apply (lambda x: isWordCapitalized(x["L"]), axis = 1)

train_2["L Captilaized"] = train_2.apply (lambda x: isWordCapitalized(x["L"]), axis = 1)
test_2["L Captilaized"] = test_2.apply (lambda x: isWordCapitalized(x["L"]), axis = 1)

In [43]:
# Feature 5
# If R is Capitalized
train_1["R Captilaized"] = train_1.apply (lambda x: isWordCapitalized(x["R"]), axis = 1)
test_1["R Captilaized"] = test_1.apply (lambda x: isWordCapitalized(x["R"]), axis = 1)

train_2["R Captilaized"] = train_2.apply (lambda x: isWordCapitalized(x["R"]), axis = 1)
test_2["R Captilaized"] = test_2.apply (lambda x: isWordCapitalized(x["R"]), axis = 1)

In [44]:
train_1.drop(train_1[train_1['labels'] == "TOK"].index, inplace = True) 
test_1.drop(test_1[test_1['labels'] == "TOK"].index, inplace = True) 

train_2.drop(train_2[train_2['labels'] == "TOK"].index, inplace = True) 
test_2.drop(test_2[test_2['labels'] == "TOK"].index, inplace = True) 

train_3.drop(train_3[train_3['labels'] == "TOK"].index, inplace = True) 
test_3.drop(test_3[test_3['labels'] == "TOK"].index, inplace = True) 

,Words,labels
1,Mr.,NEOS
24,Japan.,EOS
40,says.,EOS
56,Mr.,NEOS
61,changes.,EOS
...,...,...
148527,conviction.,EOS
148572,appeals.,EOS
148593,inmates.,EOS
148622,states.,EOS


In [45]:
def ifQuotationMark (entry):
    if isinstance(entry, str) :
        entry = entry.strip()
        if entry == "\"" or entry == "\'":
            return True
    return False

In [46]:
# Feature 6
# Is R a " or '?
train_1["R Quote"] = train_1.apply (lambda x: ifQuotationMark(x["R"]), axis = 1)
test_1["R Quote"] = test_1.apply (lambda x: ifQuotationMark(x["R"]), axis = 1)

train_3["R Quote"] = train_1.apply (lambda x: ifQuotationMark(x["R"]), axis = 1)
test_3["R Quote"] = test_1.apply (lambda x: ifQuotationMark(x["R"]), axis = 1)

In [47]:
# Feature 7
# Is L a " or '?
train_1["L Quote"] = train_1.apply (lambda x: ifQuotationMark(x["L"]), axis = 1)
test_1["L Quote"] = test_1.apply (lambda x: ifQuotationMark(x["L"]), axis = 1)

train_3["L Quote"] = train_1.apply (lambda x: ifQuotationMark(x["L"]), axis = 1)
test_3["L Quote"] = test_1.apply (lambda x: ifQuotationMark(x["L"]), axis = 1)

In [48]:
def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

In [49]:
# Feature 8
# Is R a digit?
train_1["R Digit"] = train_1.apply (lambda x: is_number(x["R"]), axis = 1)
test_1["R Digit"] = test_1.apply (lambda x: is_number(x["R"]), axis = 1)

train_3["R Digit"] = train_1.apply (lambda x: is_number(x["R"]), axis = 1)
test_3["R Digit"] = test_1.apply (lambda x: is_number(x["R"]), axis = 1)

In [50]:
y_train_1 = train_1.labels
x_train_1 = train_1.drop(["Words", "labels"], axis = 1)

x_test_1 = test_1.drop(["Words", "labels"], axis = 1)
y_test_1 = test_1.labels

y_train_2 = train_2.labels
x_train_2 = train_2.drop(["Words", "labels"], axis = 1)

x_test_2 = test_2.drop(["Words", "labels"], axis = 1)
y_test_2 = test_2.labels

y_train_3 = train_3.labels
x_train_3 = train_3.drop(["Words", "labels"], axis = 1)

x_test_3 = test_3.drop(["Words", "labels"], axis = 1)
y_test_3 = test_3.labels

In [51]:
le = LabelEncoder()
x_train_1["L"] = x_train_1["L"].astype(str)
x_train_1["R"] = x_train_1["R"].astype(str)

x_train_1["L"] = le.fit_transform(x_train_1["L"])
x_train_1["R"] = le.fit_transform(x_train_1["R"])

x_train_2["L"] = x_train_2["L"].astype(str)
x_train_2["R"] = x_train_2["R"].astype(str)

x_train_2["L"] = le.fit_transform(x_train_2["L"])
x_train_2["R"] = le.fit_transform(x_train_2["R"])


In [52]:
le = LabelEncoder()
x_test_1["L"] = x_test_1["L"].astype(str)
x_test_1["R"] = x_test_1["R"].astype(str)

x_test_1["L"] = le.fit_transform(x_test_1["L"])
x_test_1["R"] = le.fit_transform(x_test_1["R"])

x_test_2["L"] = x_test_2["L"].astype(str)
x_test_2["R"] = x_test_2["R"].astype(str)

x_test_2["L"] = le.fit_transform(x_test_2["L"])
x_test_2["R"] = le.fit_transform(x_test_2["R"])

In [53]:
Clf = DecisionTreeClassifier(criterion = "entropy")
Clf = Clf.fit(x_train_1, y_train_1)
y_pred_1 = Clf.predict(x_test_1)

In [54]:
print("Accuracy for the 8 features:",metrics.accuracy_score(y_test_1, y_pred_1)*100)

Accuracy for the 8 features: 89.3667861409797


In [55]:
Clf = DecisionTreeClassifier(criterion = "entropy")
Clf = Clf.fit(x_train_2, y_train_2)
y_pred_2 = Clf.predict(x_test_2)

In [56]:
print("Accuracy for the 5 features:",metrics.accuracy_score(y_test_2, y_pred_2)*100)

Accuracy for the 5 features: 89.06810035842294


In [57]:
Clf = DecisionTreeClassifier(criterion = "entropy")
Clf = Clf.fit(x_train_3, y_train_3)
y_pred_3 = Clf.predict(x_test_3)

In [58]:
print("Accuracy for the 3 features:",metrics.accuracy_score(y_test_3, y_pred_3)*100)

Accuracy for the 3 features: 76.88172043010752
